# About

Comparing theoretical analysis with numeric results in a simplified 2-dimensional Kuramoto system with adaptive delays and no
self-feedback interactions.

# Library

In [ ]:
from __future__ import division, print_function

import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import math
from math import pi

from lib import *

from IPython.display import clear_output
%matplotlib inline

# Directories

In [ ]:
# Data files
name_folder = 'matlab2D_1'
name_file = 'gain_30_num_1'

# Directory
dir_main = os.getcwd()
dir_folder = os.path.join(dir_main, 'data', name_folder)
dir_mat = os.path.join(dir_folder, name_file)

# Import using Scipy
dict_mat = sio.loadmat(dir_mat)